<div style="display: table; width: 100%;">
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 70%;">
    <h1>Visualización Avanzada de Datos en Data Science</h1>
  </div>
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 30%;">
    <img src="https://raw.githubusercontent.com/UIDE-Tareas/3-Visualizacion-Avanzada-Datos-Data-Science-Tarea2/refs/heads/main/Assets/UideLogo.png" alt="Texto alternativo" style="width:50%;">
  </div>
</div>
<hr />

### 🟦 Componente Práctico 2   
🟡 Grupo: 4      
🟡 Semana: 2      
🟡 Docente: Edwin Jahir Rueda Rojas(edruedaro@uide.edu.ec)     

### 🟦 Realizado por:   
Estudiantes

💻 Diego Fernando Chimbo Yepez   

💻 Hugo Javier Erazo Granda

💻 José Manuel Espinoza Bone

### 🟦 Código fuente
[https://github.com/UIDE-Tareas/3-Visualizacion-Avanzada-Datos-Data-Science-Tarea2](https://github.com/UIDE-Tareas/3-Visualizacion-Avanzada-Datos-Data-Science-Tarea2)

# FASE 0️⃣ PREPARAR EL AMBIENTE

Aquí instalamos las dependencias externas que no se encuentran en la biblioteca estándar de Python para lograr que el presente notebook se puede ejecutar sin problemas, aparte se definen algunas funciones para usar a lo largo del notebook.

In [1]:
import sys
import subprocess
import argparse
import os
from pathlib import Path

LIBS = [
    "requests",
    "numpy",
    "pandas",
    "seaborn",
    "matplotlib",
    
]

def RunCommand(commandList: list[str], printCommand: bool = True, printError:bool=True) -> subprocess.CompletedProcess:
    print("⏳", " ".join(commandList))
    stdOutput = None if printCommand else subprocess.DEVNULL
    errorOutput = None if printError else subprocess.PIPE
    result = subprocess.run(commandList,stdout=stdOutput, stderr=errorOutput, text=True)
    if result.returncode != 0 and printError:
        print(result.stderr) 
    return result

def ShowEnvironmentInfo():
    print("ℹ️  Environment Info:")
    print("Python Version:", sys.version)
    print("Platform:", sys.platform)
    print("Executable Path:", sys.executable)
    print("Current Working Directory:", os.getcwd())
    print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV"))
    print("sys.prefix:", sys.prefix)
    print("sys.base_prefix:", sys.base_prefix)

def InstallDeps():
    print("ℹ️ Installing deps.")
    RunCommand([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], printCommand=True) 
    RunCommand([sys.executable, "-m", "pip", "install", *LIBS], printCommand=True) 
    print("Deps installed.")

ShowEnvironmentInfo()
InstallDeps()

import numpy as np
import pandas as pd
import pandas
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import zipfile

pandas.set_option("display.max_rows", None)     # Muestra todas las filas
pandas.set_option("display.max_columns", None)

# Función para mostrar mensajes de información.
def ShowInfoMessage(message: str):
    display()
    display(f"ℹ️ {message}".upper())

# Función para mostrar la información del DataFrame.
def ShowTableInfo(df:pandas.DataFrame, title):
    display(f"ℹ️ {title} ℹ️".upper())
    df.info()
    display()

# Función para mostrar las n primeras filas del DataFrame.
def ShowTableHead(df:pandas.DataFrame, title:str, headQty=10):
    display(f"ℹ️ {title.upper()}: Primeros {headQty} elementos.")
    display(df.head(headQty))
    display()

# Función para mostrar las n últimas filas del DataFrame.
def ShowTableTail(df:pandas.DataFrame,title:str ,tailQty=10):
    display(f"ℹ️ {title.upper()}: Últimos {tailQty} elementos.")
    display(df.tail(tailQty))
    display()

# Mostrar el tamaño del DataFrame
def ShowTableShape(df:pandas.DataFrame, title:str):
    display(f"ℹ️ {title.upper()} - Tamaño de los datos")
    display(df.shape)
    display()

# Función para mostrar la estadística descriptiva de todas las columnas del DataFrame, por tipo de dato.
def ShowTableStats(df: pandas.DataFrame, title:str = ""):
    display(f"ℹ️ Estadística descriptiva - {title}".upper())
    numeric_types = ['int64', 'float64', 'Int64', 'Float64']
    numeric_cols = df.select_dtypes(include=numeric_types)
    if not numeric_cols.empty:
        display("    🔢 Columnas numéricas")
        numeric_desc = numeric_cols.describe().round(2).T  # Transpuesta para añadir columna
        numeric_desc["var"] = numeric_cols.var().round(2)  # Añadir varianza
        display(numeric_desc.T) 
    non_numeric_types = ['object', 'string', 'bool', 'category']
    non_numeric_cols = df.select_dtypes(include=non_numeric_types)
    if not non_numeric_cols.empty:
        display("    🔡 Columnas no numéricas")
        non_numeric_desc = non_numeric_cols.describe()
        display(non_numeric_desc)
    datetime_cols = df.select_dtypes(include=['datetime'])
    if not datetime_cols.empty:
        display("    📅 Columnas fechas")
        datetime_desc = datetime_cols.describe()
        display(datetime_desc)

# Función para mostrar los valores nulos o NaN de cada columna en un DataFrame
def ShowNanValues(df: pandas.DataFrame):
    display(f"ℹ️ Contador de valores Nulos".upper())
    nulls_count = df.isnull().sum()
    nulls_df = nulls_count.reset_index()
    nulls_df.columns = ['Columna', 'Cantidad_Nulos']
    display(nulls_df)
    display()


# Función para descargar un archivo
def DownloadFile(uri: str, filename: str):
    display(f"🟦 Descargando \"{uri}\"")
    try:
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        respuesta = requests.get(uri)
        respuesta.raise_for_status() 
        with open(filename, 'wb') as archivo:
            archivo.write(respuesta.content)
        display(f"archivo \"{filename}\" fue descargado exitosamente.")
    except requests.exceptions.RequestException as e:
        display(f"Error downloading file: {e}")

# Función para descomprimir un archivo zip
def UnzipFile(filename: str, outputDir: str):
    display(f'🟦 Descomprimiendo "{filename}" en "{outputDir}"')
    try:
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(outputDir)
        display(f"Descomprimido en: {os.path.abspath(outputDir)}")
    except Exception as e:
        display(f"Error: {e}")

# Función para mostrar un texto con colores dependiendo del valor de la condición. Verde si es True, rojo si es False.
def PrintAssert(message: str, boolExpression: bool):
    VERDE = "\033[92m"
    ROJO = "\033[91m"
    RESET = "\033[0m"
    if boolExpression:
        print(f"{VERDE}✅ {message}{RESET}")
    else:
        print(f"{ROJO}🚫 {message}{RESET}")

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

ℹ️  Environment Info:
Python Version: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
Platform: win32
Executable Path: c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe
Current Working Directory: c:\Users\Megam\OneDrive\Escritorio\3-Visualizacion-Avanzada-Datos-Data-Science-Tarea2
VIRTUAL_ENV: None
sys.prefix: c:\Users\Megam\AppData\Local\Programs\Python\Python313
sys.base_prefix: c:\Users\Megam\AppData\Local\Programs\Python\Python313
ℹ️ Installing deps.
⏳ c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip
⏳ c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install requests numpy pandas seaborn matplotlib
Deps installed.


# FASE 1️⃣ CARGA Y LIMPIEZA DE DATOS

### Descargar y Cargar los datos
- Los datos son descargados desde un repositorio público. Cuenta con 10000+ aplicaciones de google playstore y 13 caracteristicas:
    - App: Nombre de la aplicación
    - Categoría: Categoría dentro de google playstore a la cual pertenece la aplicación.
    - Rating: Calificación promedio dada por los usuarios de google playstore
    - Reviews: Total de comentarios de la aplicación.
    - Size: Tamaño de la aplicación en Kb.
    - Installs: Total de instalaciones de la aplicación.
    - Type: si es gratuita o de pago.
    - Price: Precio de la aplicación en dólares.
    - Content Rating: clasificación de la aplicación en base a su contenido.
    - Genres: Géneros dentro de google playsotre a los que pertenece la aplicación.
    - Last Updated: Fecha de la última actualización de la aplicación.
    - Current Ver: Versión actual de la aplicación.
    - Android Ver: Versión de android que soporta la aplicación.

In [5]:
#DATASET_URI = "https://raw.githubusercontent.com/UIDE-Tareas/3-Visualizacion-Avanzada-Datos-Data-Science-Tarea1/refs/heads/main/Datasets/googleplaystore_v2.csv"

DATASET_BASE_FILENAME = "Chicago_crime_data"
DATASET_ZIP_FILE = Path(f"Data/{DATASET_BASE_FILENAME}.zip").resolve()
DATASET_CSV_FILE = Path(f"Data/{DATASET_BASE_FILENAME}.csv").resolve()
DATASET_URI = f"https://raw.githubusercontent.com/UIDE-Tareas/3-Visualizacion-Avanzada-Datos-Data-Science-Tarea2/refs/heads/main/Datasets/{DATASET_BASE_FILENAME}.zip"

DownloadFile(DATASET_URI, DATASET_ZIP_FILE)
print(DATASET_ZIP_FILE)
print(DATASET_CSV_FILE)
sys.exit()








DownloadFile(DATASET_URI, DATASET_FILE)

dfOriginal = pd.read_csv(DATASET_FILE)
data = dfOriginal.copy()

ShowTableInfo(data, "DataFrame Original")
ShowTableHead(data, "DataFrame Original", 10)
ShowTableShape(data, "DataFrame Original")

'🟦 Descargando "https://raw.githubusercontent.com/UIDE-Tareas/3-Visualizacion-Avanzada-Datos-Data-Science-Tarea2/refs/heads/main/Datasets/Chicago_crime_data.zip"'

'Error downloading file: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/UIDE-Tareas/3-Visualizacion-Avanzada-Datos-Data-Science-Tarea2/refs/heads/main/Datasets/Chicago_crime_data.zip'

C:\Users\Megam\OneDrive\Escritorio\3-Visualizacion-Avanzada-Datos-Data-Science-Tarea2\Data\Chicago_crime_data.zip
C:\Users\Megam\OneDrive\Escritorio\3-Visualizacion-Avanzada-Datos-Data-Science-Tarea2\Data\Chicago_crime_data.csv


SystemExit: 

### Limpieza y manejo de datos preliminares

In [ ]:
data = dfOriginal.copy()
# eliminar registros de las filas con celda NaN
data = data.dropna()

# Cambiar nombres de columnas para evitar espacios
data.columns = [col.strip().replace(" ", "") for col in data.columns] # "Last Updated", "Current Ver", "Android Ver"

# Corregir los tipos de datos incorrectos para las columnas
data.App = data.App.astype(pandas.StringDtype()).str.strip()
data.Category = data.Category.astype(pandas.StringDtype()).str.strip()
data.Rating = data.Rating.astype(pandas.Float64Dtype())
data.Reviews = data.Reviews.astype(pandas.Int64Dtype())
data.Size = data.Size.round().astype(pandas.Int64Dtype())
data.Installs = data.Installs.apply(lambda x: x.strip().replace("+","").replace(",","")).astype(pandas.Float64Dtype())
data.Type = data.Type.apply(lambda x: x.strip()).astype(pandas.StringDtype())
data.Price = data.Price.apply(lambda x: x.strip().replace("$","")).astype(pandas.Float64Dtype())
data.ContentRating = data.ContentRating.astype(pandas.StringDtype()).str.strip()
data.Genres = data.Genres.astype(pandas.StringDtype()).str.strip()
data.LastUpdated = pandas.to_datetime(data.LastUpdated.astype(pandas.StringDtype()), format=f"%B %d, %Y", errors="raise") 
data.CurrentVer = data.CurrentVer.astype(pandas.StringDtype()).str.strip()
data.AndroidVer = data.AndroidVer.astype(pandas.StringDtype()).str.replace("and up","").str.strip()

ShowTableInfo(data, "DataFrame sin nulos y tipos corregidos")
ShowTableHead(data, "DataFrame sin nulos y tipos corregidos")


'ℹ️ DATAFRAME SIN NULOS Y TIPOS CORREGIDOS ℹ️'

<class 'pandas.core.frame.DataFrame'>
Index: 9360 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   App            9360 non-null   string        
 1   Category       9360 non-null   string        
 2   Rating         9360 non-null   Float64       
 3   Reviews        9360 non-null   Int64         
 4   Size           9360 non-null   Int64         
 5   Installs       9360 non-null   Float64       
 6   Type           9360 non-null   string        
 7   Price          9360 non-null   Float64       
 8   ContentRating  9360 non-null   string        
 9   Genres         9360 non-null   string        
 10  LastUpdated    9360 non-null   datetime64[ns]
 11  CurrentVer     9360 non-null   string        
 12  AndroidVer     9360 non-null   string        
dtypes: Float64(3), Int64(2), datetime64[ns](1), string(7)
memory usage: 1.0 MB


'ℹ️ DATAFRAME SIN NULOS Y TIPOS CORREGIDOS: Primeros 10 elementos.'

,App,Category,Rating,Reviews,Size,Installs,Type,Price,ContentRating,Genres,LastUpdated,CurrentVer,AndroidVer
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000,10000.0,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000,500000.0,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700,5000000.0,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000,50000000.0,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800,100000.0,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5600,50000.0,Free,0.0,Everyone,Art & Design,2017-03-26,1.0,2.3
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19000,50000.0,Free,0.0,Everyone,Art & Design,2018-04-26,1.1,4.0.3
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29000,1000000.0,Free,0.0,Everyone,Art & Design,2018-06-14,6.1.61.1,4.2
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33000,1000000.0,Free,0.0,Everyone,Art & Design,2017-09-20,2.9.2,3.0
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3100,10000.0,Free,0.0,Everyone,Art & Design;Creativity,2018-07-03,2.8,4.0.3


### Controles de verificación de los datos

Sirve para asegurarnos de que nuestros datos cumplan reglas lógicas presentes en nuestros negocios o proyectos.

En el caso de este proyecto se tiene que:


1.   La valoración está entre 1 y 5 para todas las aplicaciones.
2.   El número de opiniones es menor o igual que el número de instalaciones.
3.   Las aplicaciones gratuitas no deben tener un precio superior a 0.

In [ ]:
ShowInfoMessage("Verificando datos")
result = data.Rating.between(0,5).all()
PrintAssert("Calificaciones están entre 0 y 5", result)
if not result:
    data = data[data.Rating.between(0,5)]
    result = data.Rating.between(0,5).all()
    PrintAssert("Corregido Calificaciones están entre 0 y 5", data.Rating.max()<=5 and data.Rating.min()>=0, result)

result = data.Size.min()>=0
PrintAssert("Tamaño de archivo solo valores positivos", result)
if not result:
    data = data[data.Size>=0]
    result = data.Size.min()>=0
    PrintAssert("Corregido Tamaño de archivo solo valores positivos", result)

result = (data.Reviews <= data.Installs).all()
PrintAssert("Las Reviews deben ser menor al número de instalaciones", result)
if not result:
    data = data[data.Reviews <= data.Installs]
    result = (data.Reviews <= data.Installs).all()
    PrintAssert("===> 🔨 Las Reviews deben ser menor al número de instalaciones. FIXED.", result)

result = (data[data.Type == "Free"].Price == 0).all()
PrintAssert("Las aplicaciones gratuitas deben tener precio 0", result)
if not result:
    data.loc[data.Type == "Free", "Price"] = 0
    result = (data[data.Type == "Free"].Price == 0).all()
    PrintAssert("Corregido Las aplicaciones gratuitas deben tener precio 0", result)

ShowTableShape(data, "Data verificado")

'ℹ️ VERIFICANDO DATOS'

✅ Calificaciones están entre 0 y 5
✅ Tamaño de archivo solo valores positivos
🚫 Las Reviews deben ser menor al número de instalaciones
✅ ===> 🔨 Las Reviews deben ser menor al número de instalaciones. FIXED.
✅ Las aplicaciones gratuitas deben tener precio 0


'ℹ️ DATA VERIFICADO - Tamaño de los datos'

(9353, 13)